# Iceberg on EMR Features

Configure Spark session to use Iceberg.

In [ ]:
%%configure -f
{
"conf":{
    "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.catalog.spark_catalog":"org.apache.iceberg.spark.SparkSessionCatalog",
    "spark.sql.catalog.spark_catalog.type":"hive",
    "spark.sql.catalog.dev":"org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.dev.type":"hadoop",
    "spark.sql.catalog.dev.warehouse":"s3://mrworkshop-youraccountID-dayone/iceberg/catalog/tables/"
    }
}

Read some data 

In [ ]:
import org.apache.spark.sql.types._

val supplierSchema = StructType(Array(
  StructField("s_suppkey", IntegerType, true),
  StructField("s_name", StringType, true),
  StructField("s_address", StringType, true),
  StructField("s_nationkey", IntegerType, true),
  StructField("s_phone", StringType, true),
  StructField("s_acctbal", DoubleType, true),
  StructField("s_comment", StringType, true)
  )
)

val supplierDF = spark.read.schema(supplierSchema).option("delimiter","|").csv("s3://redshift-downloads/TPC-H/2.18/3TB/supplier/")

supplierDF.count()

supplierDF.show(5)

In [ ]:
%%sql 

drop table dev.db.iceberg_supplier_table

### Create Iceberg Table

In [ ]:
%%sql

CREATE TABLE dev.db.iceberg_supplier_table (
  `s_suppkey` bigint, 
  `s_name` string, 
  `s_address` string, 
  `s_nationkey` bigint, 
  `s_phone` string, 
  `s_acctbal` double, 
  `s_comment` string)
USING iceberg
partitioned by (s_nationkey)
location  's3://mrworkshop-youraccountID-dayone/iceberg/catalog/tables/db/iceberg_supplier_table'

### Insert Data into Iceberg table

In [ ]:
supplierDF.sortWithinPartitions("s_nationkey").writeTo("dev.db.iceberg_supplier_table").overwritePartitions()

In [ ]:
%%sql

select * from dev.db.iceberg_supplier_table limit 5

In [ ]:
%%sql 

select * from dev.db.iceberg_supplier_table where s_nationkey = 6 limit 5

### Update Records in Iceberg table

In [ ]:
%%sql 

UPDATE dev.db.iceberg_supplier_table
SET s_comment = 'updated'
WHERE s_nationkey = 6

In [ ]:
%%sql 

-- Notice the change in s_comment column

select * from dev.db.iceberg_supplier_table where s_nationkey = 6 limit 5

In [ ]:
%%sql 

select * from dev.db.iceberg_supplier_table where s_nationkey = 8 limit 5

### Delete records from Iceberg table

In [ ]:
%%sql

DELETE FROM dev.db.iceberg_supplier_table WHERE s_nationkey = 8

In [ ]:
%%sql 

select * from dev.db.iceberg_supplier_table where s_nationkey = 8 limit 5

### Time Travel with Iceberg 

In [ ]:
%%sql

SELECT * FROM dev.db.iceberg_supplier_table.snapshots

Time travel to the first snapshot

**REPLACE** snapshot ID from the above result (first snapshot_id)

In [ ]:
// 
val prevSupplierDF = (spark.read
    .option("snapshot-id", "7227467235143292131")
    .format("iceberg")
    .load("s3://mrworkshop-youraccountID-dayone/iceberg/catalog/tables/db/iceberg_supplier_table"))

prevSupplierDF.registerTempTable("iceberg_supplier_table_previous")

In [ ]:
%%sql 

-- This will display records we deleted 

select * from iceberg_supplier_table_previous where s_nationkey = 8 limit 5

In [ ]:
%%sql 

-- This will display the records prior to UPDATE query on s_comment column

select * from iceberg_supplier_table_previous where s_nationkey = 6 limit 5

### Incremental Queries with Iceberg

In [ ]:
%%sql

SELECT * FROM dev.db.iceberg_supplier_table.snapshots

Get the data added after start-snapshot-id until end-snapshot-id

**REPLACE** 7227467235143292131 (start-snapshot-id) and 1006091418634166352 (end-snapshot-id) with the above result (first and second snapshot_ids)

In [ ]:
val incrSupplierDF = (spark.read
    .option("start-snapshot-id", "7227467235143292131")
    .option("end-snapshot-id", "1006091418634166352")
    .format("iceberg")
    .load("s3://mrworkshop-youraccountID-dayone/iceberg/catalog/tables/db/iceberg_supplier_table"))

incrSupplierDF.registerTempTable("iceberg_supplier_table_incremental")

In [ ]:
%%sql 

-- Deleted records are displayed using incremental view 

select * from iceberg_supplier_table_previous where s_nationkey = 8 limit 5

### Inspecting Iceberg tables

Iceberg provides several user friendly options to descriptively inspect your table metadata

In [ ]:
%%sql

-- Shows table history 

SELECT * FROM dev.db.iceberg_supplier_table.history

In [ ]:
%%sql 

-- Shows table's data files 

SELECT * FROM dev.db.iceberg_supplier_table.files limit 2

In [ ]:
%%sql

-- Shows table manifests

SELECT * FROM dev.db.iceberg_supplier_table.manifests limit 2

In [ ]:
%%sql 

-- You can even join two system tables. For example, this query will show table history, with the application ID that wrote each snapshot:

select
    h.made_current_at,
    s.operation,
    h.snapshot_id,
    h.is_current_ancestor,
    s.summary['spark.app.id']
from dev.db.iceberg_supplier_table.history h
join dev.db.iceberg_supplier_table.snapshots s
  on h.snapshot_id = s.snapshot_id
order by made_current_at

### Schema evolution 

Iceberg allows for safe schema evolution and is a great option for long-term maintenance of large tables with many number of partitions 

In [ ]:
spark.table("dev.db.iceberg_supplier_table").printSchema()

In [ ]:
%%sql 

-- Iceberg allows any field to be renamed

ALTER TABLE dev.db.iceberg_supplier_table RENAME COLUMN s_comment TO comment

In [ ]:
%%sql

--Drop columns

ALTER TABLE dev.db.iceberg_supplier_table DROP COLUMN s_address

In [ ]:
%%sql

ALTER TABLE dev.db.iceberg_supplier_table
ADD COLUMNS (
    new_column string
  )

In [ ]:
spark.table("dev.db.iceberg_supplier_table").printSchema()

### Advanced: S3 Traffic Shaping with Iceberg

Those of you who run massive distributed workloads against S3 data lake may have encountered S3 bottlenecks such as S3 503 slow down errors. Iceberg provides a feature called [Object Store File Layout](https://iceberg.apache.org/docs/latest/aws/#object-store-file-layout) which is useful for traffic scaling to mitigate S3 bottlenecks. 

In [ ]:
%%sql

CREATE TABLE dev.db.iceberg_supplier_table_s3_shaping (
  `s_suppkey` bigint, 
  `s_name` string, 
  `s_address` string, 
  `s_nationkey` bigint, 
  `s_phone` string, 
  `s_acctbal` double, 
  `s_comment` string)
USING iceberg
OPTIONS ( 'write.object-storage.enabled'=true,
          'write.object-storage.path'='s3://mrworkshop-youraccountID-dayone-iceberg/')
partitioned by (s_nationkey)
location  's3://mrworkshop-youraccountID-dayone/iceberg/catalog/tables/db/iceberg_supplier_table_s3_shaping'

In [ ]:
supplierDF.sortWithinPartitions("s_nationkey").writeTo("dev.db.iceberg_supplier_table_s3_shaping").overwritePartitions()

List the object storage S3 path.

```
$ aws s3 ls s3://mrworkshop-youraccountID-dayone-iceberg/ | head
                           PRE 0009e927/
                           PRE 002341d4/
                           PRE 0034e34f/
                           PRE 0039c84a/
                           PRE 004a1b12/
                           PRE 0058a246/
                           PRE 0062d53a/
                           PRE 0081a04b/
                           PRE 008dc473/
                           PRE 00912343/

```

**REPLACE** prefix values based on the above output

```

$ aws s3 ls s3://mrworkshop-youraccountID-dayone-iceberg/0009e927 --recursive
2022-03-21 06:40:07     933324 0009e927/db/iceberg_supplier_table_s3_shaping/s_nationkey=8/00029-1303-cfffcaef-a308-4f0e-b77d-e31212afa0fb-00009.parquet


$ aws s3 ls s3://mrworkshop-youraccountID-dayone-iceberg/002341d4/ --recursive
2022-03-21 06:40:24     966751 002341d4/db/iceberg_supplier_table_s3_shaping/s_nationkey=18/00059-1333-17a7dcc5-7c4b-4e31-80fe-a08044b9844e-00019.parquet                        
                           
```

In [ ]:
%%sql 

select * from dev.db.iceberg_supplier_table_s3_shaping limit 10

In [ ]:
%%sql 

select * from dev.db.iceberg_supplier_table_s3_shaping.history limit 10